# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [38]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [39]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [37]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [4]:
!mkdir data

mkdir: no se puede crear el directorio «data»: El archivo ya existe


In [5]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31287    0 31287    0     0  46631      0 --:--:-- --:--:-- --:--:-- 46697


In [6]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70146    0 70146    0     0  87879      0 --:--:-- --:--:-- --:--:-- 87792


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [40]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [41]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [42]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [43]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [44]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 14, relationships: 70)

We can save and load our knowledge graphs as follows.

In [45]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 70)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [46]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [47]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

- What are the three types of query synthesizers doing? Describe each one in simple terms. 

    - Single-Hop Queries:
        - **Single-hop query** is a straightforward fact-based question that can be answered by retrieving information from a **single document or source**.

        - **Specific**: The answer can be **directly extracted** from one source without requiring additional reasoning.


    - Multi-Hop Queries:
        - **Multi-hop query** involves **multiple reasoning steps** to get the answer. It requires pulling information from **two or more sources**.

            - **Specific**: Each fact in the question requires retrieval from **different documents** to construct a complete answer.
            - **Abstract**: The answer is **not just a fact aggregation**, but rather an **interpretation or synthesis** of multiple sources to provide insight.




Finally, we can use our `TestSetGenerator` to generate our testset!

In [48]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Wht is Meta's role in the development of LLMs?,[Code may be the best application The ethics o...,"Meta released Llama, which allowed LLMs to be ...",single_hop_specifc_query_synthesizer
1,Does ChatGPT get lazy in December?,[Based Development As a computer scientist and...,The honest answer is 'maybe'! No-one is entire...,single_hop_specifc_query_synthesizer
2,What is the significance of GPT-4 in 2023 acco...,[Simon Willison’s Weblog Subscribe Stuff we fi...,GPT-4 is highlighted as part of the breakthrou...,single_hop_specifc_query_synthesizer
3,"what google doin with AI and LLMs, like what t...",[easy to follow. The rest of the document incl...,The leaked Google document titled 'We Have No ...,single_hop_specifc_query_synthesizer
4,What advancements have been made with Claude 3...,[Prompt driven app generation is a commodity a...,"Anthropic's Claude 3 series launched in March,...",single_hop_specifc_query_synthesizer
5,How did technological advancements in 2024 imp...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, technological advancements significan...",multi_hop_abstract_query_synthesizer
6,How have technological advancements in 2024 in...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, technological advancements significan...",multi_hop_abstract_query_synthesizer
7,How have advancements in AI agents and real-ti...,[<1-hop>\n\nPrompt driven app generation is a ...,Recent advancements in AI have seen the integr...,multi_hop_abstract_query_synthesizer
8,How do the challenges of evaluating LLMs and t...,[<1-hop>\n\n“agents” as dependent on AGI itsel...,"The challenges of evaluating LLMs, as describe...",multi_hop_specific_query_synthesizer
9,How did the advancements in Large Language Mod...,[<1-hop>\n\neasy to follow. The rest of the do...,"In 2023, Large Language Models (LLMs) marked a...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [49]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [50]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What are some key developments in the field of...,[Code may be the best application The ethics o...,"In 2024, it was discovered that Large Language...",single_hop_specifc_query_synthesizer
1,What role does the ChatGPT Code Interpreter pl...,[Based Development As a computer scientist and...,The ChatGPT Code Interpreter allows the LLM to...,single_hop_specifc_query_synthesizer
2,What AI mean?,[Simon Willison’s Weblog Subscribe Stuff we fi...,AI refers to the latest and most interesting d...,single_hop_specifc_query_synthesizer
3,Wht r the ethical concerns surounding the use ...,[easy to follow. The rest of the document incl...,The ethical concerns surrounding the use of AI...,single_hop_specifc_query_synthesizer
4,How has the environmental impact of AI models ...,[<1-hop>\n\nPrompt driven app generation is a ...,The environmental impact of AI models has impr...,multi_hop_abstract_query_synthesizer
5,How has the shift from universal access to sub...,[<1-hop>\n\nPrompt driven app generation is a ...,The shift from universal access to subscriptio...,multi_hop_abstract_query_synthesizer
6,How has the environmental impact of AI models ...,[<1-hop>\n\nPrompt driven app generation is a ...,The environmental impact of AI models has impr...,multi_hop_abstract_query_synthesizer
7,How has the environmental impact of AI models ...,[<1-hop>\n\nPrompt driven app generation is a ...,The environmental impact of AI models has seen...,multi_hop_abstract_query_synthesizer
8,How do the challenges of 'Vibes Based Developm...,[<1-hop>\n\nBased Development As a computer sc...,The challenges of 'Vibes Based Development' an...,multi_hop_specific_query_synthesizer
9,How have the advancements in multi-modal LLMs ...,[<1-hop>\n\ngets you OpenAI’s most expensive m...,"The advancements in multi-modal LLMs, such as ...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [52]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [53]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [54]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [55]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [56]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [57]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [58]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [59]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [60]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [61]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [62]:
rag_chain.invoke({"question" : "What are Agents?"})

'Agents are an infuriatingly vague term that generally refers to AI systems that can act on behalf of a user. There are different interpretations of "agents," with some people seeing them as entities that perform tasks like a travel agent, while others view them as LLMs (large language models) that have access to tools to solve problems. The term lacks a clear and consistent definition, leading to confusion about what it truly means.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [63]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [64]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: Instructs an llm to directly grade a response as "correct" or "incorrect" based on the reference answer.
- `labeled_helpfulness_evaluator`: Evaluates if the answer is helpful to the user based in a reference answer. Goes through an step by step reasoning to determine that and end up answering Y (yes) or N (no).
- `dope_or_nope_evaluator`: Evaluates if the answer is dope, lit or cool to the user based in a reference answer. Goes through an step by step reasoning to determine that and end up answering Y (yes) or N (no).

## LangSmith Evaluation

In [65]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'worthwhile-comparison-56' at:
https://smith.langchain.com/o/59595845-78fe-49c0-8baf-8549ccb8fe48/datasets/9eca0bfe-be34-4bec-a3df-ad0ec7868ada/compare?selectedSessions=6f3cd52a-88ee-4a55-aa34-46fa782db097




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How does the development of Claude 3 by Anthro...,The development of Claude 3 by Anthropic refle...,None,"The development of Claude 3 by Anthropic, rele...",1,0,0,2.966136,2c5a08ba-adeb-43d1-b553-fbc74919891b,119d5a13-fc95-407b-a6d4-8be5a566287e
1,"In the context of AI ethics, how has the devel...",The development and accessibility of GPT-4 and...,None,"In 2023, the development and accessibility of ...",0,0,0,3.179559,6a736a88-59a4-4cc5-a5d3-615a9a23b19f,4ff2b52d-d541-4670-b25c-de40ef27fb1d
2,How have the advancements in multi-modal LLMs ...,The advancements in multi-modal LLMs and the r...,None,"The advancements in multi-modal LLMs, such as ...",1,0,0,4.364671,7cb86317-b277-4c9f-bec0-0c9382428163,4acc0394-82dd-4d12-ae09-2c4ed2dc5b37
3,How do the challenges of 'Vibes Based Developm...,I don't know.,None,The challenges of 'Vibes Based Development' an...,0,0,0,2.457723,66e085fd-be60-432f-9f2f-3becebd4e935,02d6d575-ad43-4fab-86ea-488e2a9eb1a0
4,How has the environmental impact of AI models ...,The environmental impact of AI models has show...,None,The environmental impact of AI models has seen...,1,1,0,4.557150,0b88d2d9-6a8c-4e98-b04a-4fb8edb28588,06601d61-fbba-44a7-bccc-f6cdb2eb466e
5,How has the environmental impact of AI models ...,The environmental impact of AI models has show...,None,The environmental impact of AI models has impr...,1,0,0,3.579729,0b8c0b07-3346-4a5b-8742-3223d8ff1da4,e06c6b51-5821-4043-b6d3-41e28b62b222
6,How has the shift from universal access to sub...,The shift from universal access to subscriptio...,None,The shift from universal access to subscriptio...,1,1,0,2.561891,45acadee-c761-40e8-a8a5-6d21f8ebbfa5,40724a4d-01de-471b-b4bf-d4e89171203f
7,How has the environmental impact of AI models ...,The environmental impact of AI models has impr...,None,The environmental impact of AI models has impr...,1,0,0,3.172776,69b9761a-5261-47e7-bc0d-f1bf0234f65f,d3e63c47-85ca-4fd8-ad74-72efd34595f4
8,Wht r the ethical concerns surounding the use ...,The ethical concerns surrounding the use of AI...,None,The ethical concerns surrounding the use of AI...,1,1,0,2.927638,4e2b1fc2-6304-4d91-b5a1-b306eeeafafa,47ec75c3-cfe3-4b63-a967-3d0f76808821
9,What AI mean?,I don't know.,None,AI refers to the latest and most interesting d...,0,0,0,1.389136,29bd1ff5-2863-4eb8-a012-9d849e44faba,3340c1c0-91d7-410c-80ef-880fa3c7d55e


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [66]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [67]:
rag_documents = docs

In [68]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

-  Why would modifying our chunk size modify the performance of our application?
   - In our particular case as we not define a large number of k in the retriever we use the default, have larger chunks is benefitial.
   - This leads to fewer context breaks, improving:
     - Comprehension (LLM sees more complete thoughts).
     - Answer relevance (less chance of missing crucial information).
     - Efficiency (fewer calls to the retriever & LLM).
         


In [69]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

- Why would modifying our embedding model modify the performance of our application?
    -   Because `text-embedding-3-large`  performs better than `text-embedding-3-small` at different benchmarks and captures deeper semantic relationships.
    - Have more accurate retrieval, better generalization but also higher computational cost, slower inference speed and more expensive api calls.
    - More info: [OpenAI Embeddings][embeddings].

[embeddings]: https://openai.com/index/new-embedding-models-and-api-updates/



In [70]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [71]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [72]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [73]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Yo, agents are a bit of a mixed bag, my friend! They’re often thought of as AI systems that act on your behalf, kind of like a travel agent. But the term is super vague, and different folks throw around a ton of definitions without really pinning it down. Some see them as LLMs equipped with tools to solve problems, while others are just scratching their heads wondering when these agents will actually do something real. Right now, they have this aura of "coming soon," but there\'s skepticism about how effective they can be, especially with their tendency to believe anything they’re told. Bottom line: it’s all hype until they can tackle the whole gullibility issue!'

Finally, we can evaluate the new chain on the same test set!

In [74]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'dependable-zinc-69' at:
https://smith.langchain.com/o/59595845-78fe-49c0-8baf-8549ccb8fe48/datasets/9eca0bfe-be34-4bec-a3df-ad0ec7868ada/compare?selectedSessions=fca9edbb-2029-4982-91c9-9c2bf702351c




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How does the development of Claude 3 by Anthro...,"Alright, let’s break it down! Claude 3 by Anth...",None,"The development of Claude 3 by Anthropic, rele...",1,0,1,8.089129,2c5a08ba-adeb-43d1-b553-fbc74919891b,07e27990-df3b-4b80-8a9d-212476358593
1,"In the context of AI ethics, how has the devel...","Yo, check it! In 2023, the rise of GPT-4 and a...",None,"In 2023, the development and accessibility of ...",1,0,1,4.811497,6a736a88-59a4-4cc5-a5d3-615a9a23b19f,413a42b2-b07b-411e-bcf7-1a6de5fcf8a6
2,How have the advancements in multi-modal LLMs ...,"Yo, check it out! The advancements in multi-mo...",None,"The advancements in multi-modal LLMs, such as ...",1,0,1,4.299499,7cb86317-b277-4c9f-bec0-0c9382428163,dbba7704-e300-49b9-9bbf-25e7e5f43e4f
3,How do the challenges of 'Vibes Based Developm...,"Yo, the challenges of 'Vibes Based Development...",None,The challenges of 'Vibes Based Development' an...,1,0,1,4.506218,66e085fd-be60-432f-9f2f-3becebd4e935,6975c759-e0a0-46fa-b0ff-746b9f9c9dcb
4,How has the environmental impact of AI models ...,"Yo, check it out! The environmental impact of ...",None,The environmental impact of AI models has seen...,1,1,1,5.564790,0b88d2d9-6a8c-4e98-b04a-4fb8edb28588,ec500e4c-2445-41b7-8238-a1fb4a4e869d
5,How has the environmental impact of AI models ...,"Yo, let’s break it down! The environmental imp...",None,The environmental impact of AI models has impr...,1,1,1,4.402325,0b8c0b07-3346-4a5b-8742-3223d8ff1da4,60aa06b5-5c57-473d-a5f5-2c5265ba893f
6,How has the shift from universal access to sub...,"Yo, check it out! The shift from universal acc...",None,The shift from universal access to subscriptio...,1,1,1,4.527505,45acadee-c761-40e8-a8a5-6d21f8ebbfa5,5cdbdd6e-7698-4723-ac3a-54830df588ca
7,How has the environmental impact of AI models ...,"Alright, let’s break it down, cool cat. The en...",None,The environmental impact of AI models has impr...,1,1,1,4.504496,69b9761a-5261-47e7-bc0d-f1bf0234f65f,d96a93e8-1310-4782-b062-9e894e704a60
8,Wht r the ethical concerns surounding the use ...,"Yo, let’s break it down! The ethical concerns ...",None,The ethical concerns surrounding the use of AI...,1,1,1,4.301224,4e2b1fc2-6304-4d91-b5a1-b306eeeafafa,90386a5f-b34c-4329-b269-755967c0000b
9,What AI mean?,"AI stands for Artificial Intelligence, which h...",None,AI refers to the latest and most interesting d...,1,1,1,2.865617,29bd1ff5-2863-4eb8-a012-9d849e44faba,3d4174d3-48e8-4c37-8e91-225730f5ee4f


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.


![Metrics](metrics.png)



- All the metrics improve in the second experiment i believe this is for the following reasons:
    -   Prompt: The folowing line was added: `You must answer the questions in a dope way, be cool!`, that clearly generates an improve for `Dopeness` evaluation.
    - Larger chunks: The size of the chunks double for the second experiment which helps to capture more context and lest context breaks.
    - Embeddings: Now the embeddings are generated using `text-embedding-3-large` which helps to capture more semantic meaning and improves the retrieval phase.